In [1]:
import pickle
import pandas as pd
from datetime import datetime

In [2]:
class DataExtractor:
    def __init__(self, invoice_file_path, expired_invoices_path):
        self.invoice_file_path = invoice_file_path
        self.expired_invoices_path = expired_invoices_path
        self.expired_invoices = self.load_expired_invoices()
        self.data = self.load_invoices()

    def load_expired_invoices(self):
        with open(self.expired_invoices_path, 'r') as file:
            expired_invoices = file.read().strip().split(', ')
        return set(map(int, expired_invoices))

    def load_invoices(self):
        with open(self.invoice_file_path, 'rb') as file:
            data = pickle.load(file)
        return data

    def is_valid_date(self, date_str):
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            return True
        except ValueError:
            return False

    def transform_data(self):
        records = []
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}

        for invoice in self.data:
            invoice_id = int(str(invoice['id']).replace('O', ''))  # Ensure invoice_id is int
            created_on_str = invoice['created_on']
            if not self.is_valid_date(created_on_str):
                continue  # Skip invalid dates
            created_on = pd.to_datetime(created_on_str)

            if 'items' not in invoice:
                continue  # Skip invoices without items

            invoice_total = sum(item['item']['unit_price'] * int(item['quantity']) for item in invoice['items'] if isinstance(item['quantity'], (int, str)) and str(item['quantity']).isdigit())
            is_expired = invoice_id in self.expired_invoices

            for item in invoice['items']:
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                type_str = type_conversion.get(item['item']['type'], 'Other')
                unit_price = item['item']['unit_price']
                quantity = item['quantity']
                if isinstance(quantity, str) and not quantity.isdigit():
                    continue  # Skip invalid quantity entries
                quantity = int(quantity)  # Ensure quantity is int
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total if invoice_total != 0 else 0

                records.append([
                    invoice_id, created_on, invoiceitem_id, invoiceitem_name,
                    type_str, unit_price, total_price, percentage_in_invoice, is_expired
                ])

        df = pd.DataFrame(records, columns=[
            'invoice_id', 'created_on', 'invoiceitem_id', 'invoiceitem_name',
            'type', 'unit_price', 'total_price', 'percentage_in_invoice', 'is_expired'
        ])

        df = df.astype({
            'invoice_id': 'int64',
            'created_on': 'datetime64[ns]',
            'invoiceitem_id': 'int64',
            'invoiceitem_name': 'str',
            'type': 'str',
            'unit_price': 'int64',
            'total_price': 'int64',
            'percentage_in_invoice': 'float64',
            'is_expired': 'bool'
        })

        df = df.sort_values(by=['invoice_id', 'invoiceitem_id'])
        return df

In [3]:
invoice_file_path = r"data\invoices_new.pkl"
expired_invoices_path = r"data\expired_invoices.txt"

In [4]:
data_extractor = DataExtractor(invoice_file_path, expired_invoices_path)
processed_df = data_extractor.transform_data()

In [5]:
processed_df.to_csv('processed_invoices.csv', index=False)